## Implementación de una simulación DES

En este _notebook_ nos enfocaremos en revisar una **posible** manera de implementar una simulación basada en eventos discretos.

### Eventos

Partamos por definir (programáticamente) lo que es un evento. Consideremos que:

- Un evento tiene un tiempo de ocurrencia
- Queremos que ocurra algo en ese momento
- Nos gustaría poder identificar un evento por un identificador único

Con eso, podemos construir un a clase `Evento` que tenga cada uno de esos atributos:

- `id`: nos ayudará a identificar un evento en forma única
- `tiempo`: indicará el momento en que el evento debe ejecutarse
- `función`: contendrá lo que queramos que se ejecute en ese evento

Construiremos esta clase de esta manera:

In [1]:
from itertools import count

class Evento:
    ids = count(start=0)
    
    def __init__(self, tiempo, función):
        self.id = next(self.ids)
        self.tiempo = tiempo
        self.función = función
        
    def __repr__(self):
        template = "Evento id={}, tiempo={}"
        return template.format(self.id, self.tiempo)

### Flujo general de una simulación

Ahora que tenemos eventos, intentemos construir el flujo principal de una simulación con una clase `Simulación`. Recordemos que una simulación debe tener:

- Un tiempo de inicio
- Un tiempo en que la simulación debe finalizar
- Una lista de eventos a ejecutar, ordenada por el tiempo de ocurrencia

In [2]:
import math

class Simulación:
    
    def __init__(self, tiempo_inicio=0, tiempo_fin=math.inf):
        self.tiempo_simulación = tiempo_inicio
        self.tiempo_fin = tiempo_fin
        self.eventos = []
        
    def agregar_evento(self, evento):
        """Función para agregar un evento a la cola."""
        self.eventos.append(evento)

Tenemos todos nuestros atributos básicos, pero nos falta un detalle: la lista de eventos debe mantenerse ordenada. No es materia de este curso saber cómo hacer esto de la forma más eficiente, por lo que sólo nos aseguraremos que, cada vez que se acceda a esta lista, se ordene por el tiempo de los eventos. Haremos esto vía _properties_.

In [3]:
class Simulación:
    
    def __init__(self, tiempo_inicio=0, tiempo_fin=math.inf):
        self.tiempo_simulación = tiempo_inicio
        self.tiempo_fin = tiempo_fin
        self._eventos = []
    
    @property
    def eventos(self):
        """Entrega la cola de eventos, ordenada por tiempo."""
        self._eventos.sort(key=lambda x: x.tiempo)
        return self._eventos

    def agregar_evento(self, evento):
        """Función para agregar un evento a la cola."""
        self.eventos.append(evento)

Ahora, nos falta el flujo principal. Recordemos que el algoritmo general de una simulación DES es el siguiente:

    MIENTRAS la lista de eventos no esté vacía y el tiempo de simulación no termine:
        tomar un evento desde el principio de la lista de eventos
        avanzar el tiempo de simulación al tiempo del evento
        simular el evento
        
Implementaremos este algoritmo en una función llamada `run` de nuestra clase `Simulación`:

In [4]:
class Simulación:
    
    def __init__(self, tiempo_inicio=0, tiempo_fin=math.inf):
        self.tiempo_simulación = tiempo_inicio
        self.tiempo_fin = tiempo_fin
        self._eventos = []
    
    @property
    def eventos(self):
        """Entrega la cola de eventos, ordenada por tiempo."""
        self._eventos.sort(key=lambda x: x.tiempo)
        return self._eventos

    def agregar_evento(self, evento):
        """Función para agregar un evento a la cola."""
        self.eventos.append(evento)
    
    def run(self):
        """Ejecuta la simulación."""
        # Revisamos que haya un evento, y que este evento esté dentro del límite de tiempo
        while self.eventos and self.eventos[0] < self.tiempo_fin:
            # Sacamos el primer evento de la lista
            evento = self.eventos.pop(0)
            # Avanzamos el tiempo de simulación al tiempo del evento
            self.tiempo_simulación = evento.tiempo
            # Simulamos el evento
            evento.función()

#### Simulación como entorno

Una instancia de la clase `Simulación` **debería funcionar como si fuese un _entorno_**, es decir, todos los objetos que participan en nuestro modelo deberían poder interactuar de alguna forma con la simulación, y viceversa. Eso permite, incluso, hacer que dos objetos interactúen en forma indirecta dentro de la simulación.

Podemos cumplir una parte de la interacción entregando un objeto de clase `Simulación` a cada objeto que participe en nuestro modelo. Nos falta la otra parte, ¿Cómo hacemos que la `Simulación` interactúe con los objetos? Una solución podría ser que cada objeto entrege su propia referencia a la simulación, mediante algún método. Sin embargo, esto nos conduciría a tener que _hardcodear_ cosas en nuestra clase `Simulación`.

##### Solución: Acciones

Una solución más robusta consiste en definir "acciones" dentro de nuestra simulación. Una acción es algo que uno o más objetos son capaces de realizar mediante una función. Cada objeto es responsable de registrar las acciones que es capaz de hacer. Cuando se desencadene una acción, todas las funciones que la realizan deben ser ejecutadas.

En nuestra clase `Simulación`, representaremos cada tipo de acción con un _string_. Para cada acción almacenaremos las funciones que deben ser ejecutadas en un _set_. Por lo tanto, tendremos un diccionario que va desde los _strings_ hasta _sets_ de funciones. Finalmente, tendremos un método para desencadenar una acción, que recibe el tipo de acción a gatillar y los parámetros que serán pasados a las funciones que están registradas.

In [5]:
from collections import defaultdict

class Simulación:
    
    def __init__(self, tiempo_inicio=0, tiempo_fin=math.inf):
        self.tiempo_simulación = tiempo_inicio
        self.tiempo_fin = tiempo_fin
        self.acciones = defaultdict(set)
        self._eventos = []
    
    @property
    def eventos(self):
        """Entrega la cola de eventos, ordenada por tiempo."""
        self._eventos.sort(key=lambda x: x.tiempo)
        return self._eventos

    def agregar_evento(self, evento):
        """Función para agregar un evento a la cola."""
        self.eventos.append(evento)
        
    def agregar_función_acción(self, acción, función):
        """Permite que un objeto registre que puede hacer una acción."""
        self.acciones[acción].add(función)
        
    def ejecutar_acción(self, acción, *args, **kwargs):
        """Gatilla una acción, ejecutando todos los métodos registrados."""
        for función in self.acciones[acción]:
            función(*args, **kwargs)
        
    def run(self):
        """Ejecuta la simulación."""
        # Revisamos que haya un evento, y que este evento esté dentro del límite de tiempo
        while self.eventos and self.eventos[0].tiempo < self.tiempo_fin:
            # Sacamos el primer evento de la lista
            evento = self.eventos.pop(0)
            # Avanzamos el tiempo de simulación al tiempo del evento
            self.tiempo_simulación = evento.tiempo
            # Simulamos el evento
            evento.función()

#### Impresión de estadísticas de la simulación

Cuando simulamos un proceso, nos interesa obtener estadísticas de lo que realmente ocurrió. Además, nos gustaría saber lo que está pasando en cada momento.

Para ello, podemos implementar un método `print` para imprimir en pantalla que lo puedan ocupar todos los objetos de la simulación. Esto, con el fin de centralizar el formato con el que se imprime.

Para "avisar" a los objetos que impriman sus resultados al final de la simulación, podemos gatillar una acción al final del método `run`. Así, todo objeto que haya declarado que puede imprimir estadísticas, lo hará en ese momento.

Estas modificaciones dejan nuestra simulación de la siguiente manera:

In [6]:
from collections import defaultdict

class Simulación:
    
    def __init__(self, tiempo_inicio=0, tiempo_fin=math.inf):
        self.tiempo_simulación = tiempo_inicio
        self.tiempo_fin = tiempo_fin
        self.acciones = defaultdict(set)
        self._eventos = []
    
    @property
    def eventos(self):
        """Entrega la cola de eventos, ordenada por tiempo."""
        self._eventos.sort(key=lambda x: x.tiempo)
        return self._eventos

    def agregar_evento(self, evento):
        """Función para agregar un evento a la cola."""
        self.eventos.append(evento)
        
    def agregar_función_acción(self, acción, función):
        """Permite que un objeto registre que puede hacer una acción."""
        self.acciones[acción].add(función)
        
    def ejecutar_acción(self, acción, *args, **kwargs):
        """Gatilla una acción, ejecutando todos los métodos registrados."""
        for función in self.acciones[acción]:
            función(*args, **kwargs)
        
    def run(self):
        """Ejecuta la simulación."""
        # Revisamos que haya un evento, y que este evento esté dentro del límite de tiempo
        while self.eventos and self.eventos[0].tiempo < self.tiempo_fin:
            # Sacamos el primer evento de la lista
            evento = self.eventos.pop(0)
            # Avanzamos el tiempo de simulación al tiempo del evento
            self.tiempo_simulación = evento.tiempo
            # [Opcional] imprimir qué sucede
            self.print('simulación', 'Extraído: {}'.format(evento))
            # Simulamos el evento
            evento.función()
        # Al finalizar la simulación, gatillamos la acción de imprimir estadísticas
        self.ejecutar_acción('IMPRIMIR_ESTADÍSTICAS')
            
    def print(self, contexto, *args, **kwargs):
        """Imprime durante la simulación."""
        template = "[TIEMPO: {:.02f} - CONTEXTO: {}]"
        print(template.format(self.tiempo_simulación, contexto.upper()), *args, **kwargs)

### Objetos de la simulación

Los objetos de nuestra simulación, deberían cumplir:
- Recibir una instancia de la simulación.
- Que se realicen ajustes iniciales. Esto puede incluir cualquier configuración, pero también agregar eventos iniciales y registrar las acciones que podrán ser gatilladas.

Abajo podemos ver cómo se vería – en general – una clase de nuestro modelo de simulación.

In [7]:
class ObjetoSimulación:
    
    def __init__(self, simulación, *args, **kwargs):
        self.simulación = simulación
        # Hacer lo que tengamos que hacer
        self.configurar()
        
    def configurar(self):
        """Método de configuración inicial"""
        # 1. Agregar eventos iniciales a la simulación (si es que aplica)
        evento = Evento(15, lambda: self.simulación.print("Objeto", "Hola mundo!"))
        self.simulación.agregar_evento(evento)
        # 2. Registrar acciones en la simulación
        self.simulación.agregar_función_acción('RECIBIR_COSA', self.recibir_cosas)
        self.simulación.agregar_función_acción('IMPRIMIR_ESTADÍSTICAS', self.imprimir_estadísticas)
        
    def recibir_cosas(self, cosa):
        """
        Implementa la acción de recibir una cosa.
        
        Es una acción que puede ser gatillada por cualquier
        objeto dentro de la simulación.
        """
        # Acá deberíamos procesar qué hacer con esta cosa que recibimos
        # Esto podría derivar, incluso, en gatillar más acciones, o
        # programar otro evento.
        pass
    
    def imprimir_estadísticas(self):
        """Implementa la acción de imprimir estadísticas finales"""
        print("En realidad, no hice nada :/")
    
    def hacer_otra_cosa(self):
        """Esto es otro método del objeto, y no es una acción."""
        # Podemos hacer otra cosa acá
        pass

Y como unimos todos los componentes:

In [8]:
simulación = Simulación(tiempo_fin=500)
objeto = ObjetoSimulación(simulación)

simulación.run()

[TIEMPO: 15.00 - CONTEXTO: SIMULACIÓN] Extraído: Evento id=0, tiempo=15
[TIEMPO: 15.00 - CONTEXTO: OBJETO] Hola mundo!
En realidad, no hice nada :/


## Ejemplo de revisión técnica

¡Muy bien! Al parecer tenemos todo lo necesario para volver a programar nuestra revisión técnica, pero ahora con eventos discretos. Partamos con la clase `Taller`.

In [9]:
from collections import deque
from statistics import mean

class Taller:
    """Esta clase modela el taller de revisión técnica."""
    
    def __init__(self, simulación):
        self.simulación = simulación
        self.cola = deque()
        self.tiempos_atención = []
        self.tiempos_espera = []
        self.vehículo_actual = None
        self.configurar()
        
    def configurar(self):
        # Registramos las acciones del taller 
        self.simulación.agregar_función_acción('ENCOLAR_VEH', self.encolar_vehículo)
        self.simulación.agregar_función_acción('IMPRIMIR_ESTADÍSTICAS', self.imprimir_estadísticas)

    @property
    def ocupado(self):
        return self.vehículo_actual is not None
    
    def imprimir_estadísticas(self):
        print("-" * 80)
        print("Estadísticas Taller")
        print("-" * 80)
        print("Vehículos atendidos: {}".format(len(self.tiempos_atención)))
        print("Promedio tiempo de atención: {}".format(mean(self.tiempos_atención)))
        print("Promedio tiempo de espera: {}".format(mean(self.tiempos_espera)))
        print("Vehículos todavía en cola: {}".format(len(self.cola)))
    
    def encolar_vehículo(self, vehículo):
        """Acción que ingresa un vehículo a la cola del taller."""
        self.cola.append(vehículo)
        # Imprimimos lo que sucede
        self.simulación.print("taller", "{} entró a la cola".format(vehículo))
        # Si no estamos atendiendo a ningún vehículo, atendemos a este de inmediato
        if not self.ocupado:
            self.atender_siguiente_vehículo()
            

    def atender_siguiente_vehículo(self):
        """Función para atender al siguiente vehículo de la cola."""
        # Seteamos el vehículo que estamos atendiendo ahora
        self.vehículo_actual = self.cola.popleft()
        
        # Agregamos el tiempo que el vehículo tuvo que esperar para ser atendido
        self.tiempos_espera.append(self.simulación.tiempo_simulación - \
                                   self.vehículo_actual.tiempo_creación)
        # Agregamos el tiempo que el vehículo se demorará en ser revisado
        self.tiempos_atención.append(self.vehículo_actual.tiempo_atención)
        
        # Calculamos cuando estará listo el vehículo
        tiempo_fin_atención = self.simulación.tiempo_simulación + \
                              self.vehículo_actual.tiempo_atención
        # Creamos el evento de fin de atención, donde ejecutamos terminar_vehículo
        evento = Evento(tiempo_fin_atención, self.terminar_vehículo)
        # Agregamos el evento a la simulación
        self.simulación.agregar_evento(evento)
        
        # Imprimimos lo que sucede
        self.simulación.print("taller", "{} está siendo atendido".format(self.vehículo_actual))
        
        
    def terminar_vehículo(self):
        """Función que se ejecuta al terminar de atender un vehículo."""
        self.simulación.print("taller", "{} sale".format(self.vehículo_actual))
        # Ya no estamos atendiendo ningún vehículo
        self.vehículo_actual = None
        # Vemos si hay otro vehículo, para comenzar a atenderlo
        if self.cola:
            self.atender_siguiente_vehículo()

Ahora, vamos con el vehículo:

In [10]:
from random import expovariate

class Vehículo:
    
    ids = count(start=0)
    
    def __init__(self, simulación, tipo, tasa_media_atención):
        self.id = next(self.ids)
        self.simulación = simulación
        # Colocamos 1 minuto como mínimo de tiempo de atención
        self.tiempo_atención = max(expovariate(tasa_media_atención), 1)
        self.tiempo_creación = self.simulación.tiempo_simulación
        self.tipo = tipo
        
    def __repr__(self):
        return "{} id={}".format(self.tipo.capitalize(), self.id)

Falta algo, ¿no? Tenemos que hacer "aparecer" los vehículos para que la planta los revise. Hagamos una clase especializada en eso:

In [11]:
from random import choice

class CreadorVehículos:
    
    def __init__(self, simulación, tasa_media_creación, info_vehículos):
        """
        Inicializa el creador de vehículos.
        
        info_vehículos es un diccionario de str -> float, donde las
        keys son los tipos de vehículos y 
        los valores son las tasas medias de atención de ese tipo.
        """
        self.simulación = simulación
        self.tasa_media_creación = tasa_media_creación
        self.info_vehículos = info_vehículos
        self.configurar()
        
    def configurar(self):
        # Creamos el evento de la primera creación de vehículo
        self.programar_creación_vehículo()
        
    def programar_creación_vehículo(self):
        # Vemos en cuánto tiempo (a partir de ahora) liberaremos otro vehículo
        retraso = expovariate(self.tasa_media_creación)
        # Calculamos el tiempo en que se debe liberar el vehículo
        tiempo_evento = self.simulación.tiempo_simulación + retraso
        # Creamos el evento
        evento = Evento(tiempo_evento, self.crear_vehículo)
        # Agregamos el evento a la simulación
        self.simulación.agregar_evento(evento)
        
    def crear_vehículo(self):
        # Elegimos el tipo de vehículo a crear, junto con su tasa media de atención
        tipo, tasa_media_atención = choice(list(self.info_vehículos.items()))
        # Creamos el vehículo
        vehículo = Vehículo(self.simulación, tipo, tasa_media_atención)
        # Gatillamos la acción de encolar un vehículo en la planta de revisión
        self.simulación.ejecutar_acción('ENCOLAR_VEH', vehículo)
        # Programamos la siguiente creación de vehículo
        self.programar_creación_vehículo()

Ahora, ¡a unir todo!

In [12]:
info_vehículos = {'moto': 1 / 8, 'auto': 1 / 15, 'camioneta': 1 / 20}

simulación = Simulación(tiempo_fin=50)
taller = Taller(simulación)
creador_vehículos = CreadorVehículos(simulación, 1 / 5, info_vehículos)

In [13]:
simulación.run()

[TIEMPO: 15.23 - CONTEXTO: SIMULACIÓN] Extraído: Evento id=1, tiempo=15.233222184035293
[TIEMPO: 15.23 - CONTEXTO: TALLER] Auto id=0 entró a la cola
[TIEMPO: 15.23 - CONTEXTO: TALLER] Auto id=0 está siendo atendido
[TIEMPO: 19.96 - CONTEXTO: SIMULACIÓN] Extraído: Evento id=3, tiempo=19.959551449245236
[TIEMPO: 19.96 - CONTEXTO: TALLER] Camioneta id=1 entró a la cola
[TIEMPO: 21.05 - CONTEXTO: SIMULACIÓN] Extraído: Evento id=4, tiempo=21.05114236298514
[TIEMPO: 21.05 - CONTEXTO: TALLER] Camioneta id=2 entró a la cola
[TIEMPO: 29.49 - CONTEXTO: SIMULACIÓN] Extraído: Evento id=2, tiempo=29.489892852630106
[TIEMPO: 29.49 - CONTEXTO: TALLER] Auto id=0 sale
[TIEMPO: 29.49 - CONTEXTO: TALLER] Camioneta id=1 está siendo atendido
[TIEMPO: 34.87 - CONTEXTO: SIMULACIÓN] Extraído: Evento id=5, tiempo=34.869904153937945
[TIEMPO: 34.87 - CONTEXTO: TALLER] Auto id=3 entró a la cola
[TIEMPO: 37.76 - CONTEXTO: SIMULACIÓN] Extraído: Evento id=7, tiempo=37.759077313168966
[TIEMPO: 37.76 - CONTEXTO: TALLE

Como podemos observar, la variación del tiempo de simulación depende exclusivamente de los eventos que ocurren a lo largo de la simulación. En cada iteración ingresa un vehículo a la cola de espera y se genera aleatoriamente el tiempo de llegada para el próximo vehículo (o evento). Luego, cada vez que un vehículo ingresa al taller ocurre un cambio en el estado del sistema. En este caso, el taller está ocupado, y se establece un tiempo aleatorio de atención. Mientras no salga un vehículo del taller, los vehículos entrantes se siguen acumulando en la cola de espera y el tiempo de simulación **_avanza_**. Al salir un vehículo del taller, es ese el nuevo tiempo para el sistema y genera un nuevo cambio en los estados.